## Instalação de dependências externas necessárias

In [1]:
%pip install pandas -q
%pip install matplotlib -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Carregamento das dependências

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

## Importação do dataset e criação de DB

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/LSouzaAndrade/TechChallenge3/refs/heads/main/Datasets/StudentPerformanceFactors.csv')
conn = sqlite3.connect('./Datasets/StudentPerformanceFactors.db')
# dataset.to_sql('StudentData', conn, if_exists='replace', index=False)
# conn.close()

### Data Cleaning

In [8]:
df.shape

(6607, 20)

## Features com valores unicos
Verificando a existência de features com valores únicos (devem ser dropadas por não contribuirem para a explicar a variação da feature target)

In [9]:
df.nunique().sort_values()

Extracurricular_Activities     2
Internet_Access                2
Gender                         2
School_Type                    2
Learning_Disabilities          2
Distance_from_Home             3
Parental_Involvement           3
Access_to_Resources            3
Parental_Education_Level       3
Motivation_Level               3
Family_Income                  3
Teacher_Quality                3
Peer_Influence                 3
Sleep_Hours                    7
Physical_Activity              7
Tutoring_Sessions              9
Hours_Studied                 41
Attendance                    41
Exam_Score                    45
Previous_Scores               51
dtype: int64

### NaN Analysis

In [10]:
df.isna().sum()

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality               78
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      90
Distance_from_Home            67
Gender                         0
Exam_Score                     0
dtype: int64